# Exercise 6.03: Predicting a Connect-4 Game Outcome Using Random Search from Keras Tuner 

In [ ]:
import pandas as pd

In [ ]:
file_url = 'https://raw.githubusercontent.com/PacktWorkshops'\
           '/The-TensorFlow-Workshop/master/Chapter06/dataset'\
           '/connect-4.csv'

In [ ]:
data = pd.read_csv(file_url)
data.head()

In [ ]:
target = data.pop('class')

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split\
                                   (data, target, \
                                    test_size=0.2, \
                                    random_state=42)

In [ ]:
!pip install keras-tuner
import kerastuner as kt

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
tf.random.set_seed(8)

In [ ]:
def model_builder(hp):
  model = tf.keras.Sequential()

  hp_l2 = hp.Choice('l2', values = [0.1, 0.01, 0.001, 0.0001])

  reg_fc1 = Dense(512, input_shape=(42,), activation='relu', \
                  kernel_regularizer=tf.keras.regularizers\
                                       .l2(l=hp_l2))
  reg_fc2 = Dense(512, activation='relu', \
                  kernel_regularizer=tf.keras.regularizers\
                                             .l2(l=hp_l2))
  reg_fc3 = Dense(128, activation='relu', \
                  kernel_regularizer=tf.keras.regularizers\
                                      .l2(l=hp_l2))
  reg_fc4 = Dense(128, activation='relu', \
                  kernel_regularizer=tf.keras.regularizers\
                                       .l2(l=hp_l2))
  reg_fc5 = Dense(3, activation='softmax')
  
  model.add(reg_fc1)
  model.add(reg_fc2)
  model.add(reg_fc3)
  model.add(reg_fc4)
  model.add(reg_fc5)
  
  loss = tf.keras.losses.SparseCategoricalCrossentropy()
  optimizer = tf.keras.optimizers.Adam(0.001)
  model.compile(optimizer = optimizer, loss = loss, \
                metrics = ['accuracy'])
  
  return model

In [ ]:
tuner = kt.RandomSearch(model_builder, objective='val_accuracy', \
                        max_trials=10)   

In [ ]:
tuner.search(X_train, y_train, validation_data=(X_test, y_test))

In [ ]:
best_hps = tuner.get_best_hyperparameters()[0]

In [ ]:
best_l2 = best_hps.get('l2')
best_l2

In [ ]:
model = tuner.hypermodel.build(best_hps)

In [ ]:
model.fit(X_train, y_train, epochs=5, \
          validation_data=(X_test, y_test))